### 언어 모델 선언하기

In [19]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
llm = ChatOpenAI(model="gpt-4o-mini")

llm.invoke([HumanMessage("잘 지냈어?")])

AIMessage(content='네, 잘 지냈습니다! 당신은 잘 지내고 계신가요? 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 12, 'total_tokens': 38, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CAwQpPcuifirqpS7lhtdWJniCtBGR', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--fde6fead-b9de-4647-8dd2-e26d3b39e4a4-0', usage_metadata={'input_tokens': 12, 'output_tokens': 26, 'total_tokens': 38, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### 랭체인에 시간을 파악하는 도구 추가하기

In [20]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수

    Args:
    timezone (str): 타임존(예: 'Asia/Seoul'). 실제 존재해야함
    location (str): 지역명. 타임존은 모든 지명에 대응되지 않으므로 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_locao_time = f'{timezone} ({location}) 현재 시각 {now}'

    print(location_and_locao_time)
    return location_and_locao_time

### 함수 get_current_time을 랭체인으로 llm에 연결하기

In [21]:
# 도구를 tools 리스트에 추가하고, tool_dict에도 추가
tools = [get_current_time]
tool_dict = {"get_current_time": get_current_time,}

# 도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 답변을 생성할 수 있음
llm_with_tools = llm.bind_tools(tools)

### 사용자의 질문과 도구를 사용해 언어모델 답변 생성하기

In [22]:
from langchain_core.messages import SystemMessage

# 사용자의 질문과 도구를 사용해 언어 모델 답변 생성
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇 시야?"),
]

# llm_with_tools를 사용해 사용자의 질문에 언어 모델 답변 생성
response = llm_with_tools.invoke(messages)
messages.append(response)

# 생성된 언어 모델 답변 출력
print(messages)

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇 시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yPAhThmugdYPx51EYcCcj5ad', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"부산"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 130, 'total_tokens': 153, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CAwQq10OfGxbc3uQxWi63zEx22VBO', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4e0999fa-b66e-48d1-b638-a067af3282ce-0', tool_call

### 함수 실행 결과 출력하기

In [23]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재 시각 2025-09-01 19:52:09


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇 시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yPAhThmugdYPx51EYcCcj5ad', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"부산"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 130, 'total_tokens': 153, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CAwQq10OfGxbc3uQxWi63zEx22VBO', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4e0999fa-b66e-48d1-b638-a067af3282ce-0', tool_ca

### 함수 실행 결과를 문장으로 출력하기

In [24]:
llm_with_tools.invoke(messages)

AIMessage(content='부산은 현재 2025년 9월 1일 19시 52분입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 186, 'total_tokens': 209, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CAwQsHGrgLxEXPtwf1YFbQrvI54CV', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--9f2e639a-0f05-4630-8f54-5c5193310e21-0', usage_metadata={'input_tokens': 186, 'output_tokens': 23, 'total_tokens': 209, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### pydantic으로 함수 입력값 형식 표현하기

In [25]:
from pydantic import BaseModel, Field

class StockHistoryInput(BaseModel):
    ticker: str = Field(..., title="주식 코드", description=f"주식 코드 (예: AAPL)")
    period: str = Field(..., title="기간", description="주식 데이터 조회 기간 (예: 1d, 1mo, 1y)")

### pydentic으로 함수 입력값 형식 표현하기(2)

In [26]:
import yfinance as yf

@tool
def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
    """ 주식 종목의 가격 데이터를 조회하는 함수 """
    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(period=stock_history_input.period)
    history_md = history.to_markdown()

    return history_md

tools = [get_current_time, get_yf_stock_history]
tool_dict = {"get_current_time": get_current_time, "get_yf_stock_history": get_yf_stock_history}

llm_with_tools = llm.bind_tools(tools)

### 함수 실행 결과 출력하기

In [27]:
messages.append(HumanMessage("테슬라는 한 달 전에 비해 주가가 올랐나 내렸나?"))

response = llm_with_tools.invoke(messages)
print(response)
messages.append(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_wdd6vI5oBCL03eSF4kWor6dt', 'function': {'arguments': '{"stock_history_input":{"ticker":"TSLA","period":"1mo"}}', 'name': 'get_yf_stock_history'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 277, 'total_tokens': 304, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CAwQtcgG2f3refoYN0iQ2CqkLyibC', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--37a6576c-e8d3-4b95-9f22-9a7a0a8e6909-0' tool_calls=[{'name': 'get_yf_stock_history', 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}, 'id': 'call_wdd6vI5oBCL03eSF4kWor6dt', 'type': 'tool_call'}] usage_metadata={'inp

### 결과를 메시지로 추가하기

In [28]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    print(tool_msg)

{'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}
content='| Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |\n|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|\n| 2025-07-30 00:00:00-04:00 | 322.18 | 324.45 | 311.62 |  319.04 | 8.39319e+07 |           0 |              0 |\n| 2025-07-31 00:00:00-04:00 | 319.61 | 321.37 | 306.1  |  308.27 | 8.52709e+07 |           0 |              0 |\n| 2025-08-01 00:00:00-04:00 | 306.21 | 309.31 | 297.82 |  302.63 | 8.91214e+07 |           0 |              0 |\n| 2025-08-04 00:00:00-04:00 | 309.08 | 312.12 | 303    |  309.26 | 7.86839e+07 |           0 |              0 |\n| 2025-08-05 00:00:00-04:00 | 308.95 | 312.45 | 305.5  |  308.72 | 5.79613e+07 |           0 |              0 |\n| 2025-08-06 00:00:00-04:00 | 307.89 | 320.47 | 306.93 |  319.91 | 7.85236e+07 |           0 |              0 |\n| 2025-08-07 00:00:00-04:0

### 자연어로 함수 결과 처리하기

In [29]:
llm_with_tools.invoke(messages)

AIMessage(content='한 달 전 테슬라(TSLA)의 주가는 다음과 같았습니다:\n\n- **2025년 7월 30일**: 종가 319.04\n\n현재(2025년 9월 1일) 테슬라의 주가는 351.67입니다.\n\n- **현재(2025년 9월 1일)**: 종가 351.67\n\n따라서 테슬라는 한 달 전에 비해 주가가 올랐습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 103, 'prompt_tokens': 1696, 'total_tokens': 1799, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CAwQuKDwVDqWGRCE44vSIRDDFUYus', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e7571b25-b516-4a1d-8cca-b6915942bece-0', usage_metadata={'input_tokens': 1696, 'output_tokens': 103, 'total_tokens': 1799, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})